In [16]:

import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 2000
nb_validation_samples = 400
epochs = 50
batch_size = 16


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save('bottleneck_features_train.npy',
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_validation.npy',
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load('bottleneck_features_validation.npy')
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Train on 2000 samples, validate on 400 samples
Epoch 1/50
2000/2000 [==============================] - 5s 3ms/step - loss: 0.7536 - acc: 0.7685 - val_loss: 0.4975 - val_acc: 0.7525
Epoch 2/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.3524 - acc: 0.8585 - val_loss: 0.2396 - val_acc: 0.8800
Epoch 3/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2900 - acc: 0.8800 - val_loss: 0.2205 - val_acc: 0.9025
Epoch 4/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2313 - acc: 0.9065 - val_loss: 0.3706 - val_acc: 0.8700
Epoch 5/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2241 - acc: 0.9140 - val_loss: 0.2489 - val_acc: 0.9075
Epoch 6/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2022 - acc: 0.9255 - val_loss: 0.2549 - val_acc: 0.9025
Epoch 7/50
2000/2000 [==============================] - 4s 2ms/ste